In [ ]:
!pip install flwr
!pip install -q flwr[simulation] torch torchvision matplotlib



!pip install -U flwr["simulation"]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

restart the environment


In [ ]:
import flwr as fl
import tensorflow as tf


In [ ]:
!wget https://github.com/fandulu/DD-Net/archive/master.zip
!unzip master.zip
!rm master.zip

--2023-06-23 01:48:25--  https://github.com/fandulu/DD-Net/archive/master.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/fandulu/DD-Net/zip/refs/heads/master [following]
--2023-06-23 01:48:26--  https://codeload.github.com/fandulu/DD-Net/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [        <=>         ]  79.09M  11.1MB/s    in 7.1s    

2023-06-23 01:48:33 (11.1 MB/s) - ‘master.zip’ saved [82934337]

Archive:  master.zip
f26a9994b0bafc41096fa269eab89c2757d71499
   creating: DD-Net-master/
  inflating: DD-Net-master/.gitignore  
   creating: DD-Net-master/JHMD

In [ ]:
import numpy as np
import math
import random
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import glob
from tqdm import tqdm
import pickle
import scipy.ndimage.interpolation as inter
from scipy.signal import medfilt
from scipy.spatial.distance import cdist

from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.layers.core import *
from tensorflow.keras.callbacks import *
from keras.layers.convolutional import *
from keras import backend as K
import keras
import tensorflow as tf

import google.colab.files


In [ ]:
Train = pickle.load(open("/content/DD-Net-master/data/JHMDB/GT_train_1.pkl", "rb"))
Test = pickle.load(open("/content/DD-Net-master/data/JHMDB/GT_test_1.pkl", "rb"))

In [ ]:
print(set(Train['label']))
print(len(set(Train['label'])))

{'pick', 'clap', 'jump', 'run', 'push', 'stand', 'pullup', 'throw', 'walk', 'catch', 'pour', 'wave', 'sit', 'golf'}
14


In [ ]:
# Split the dictionary into separate lists
pose_values = Train['pose']
label_values = Train['label']

train_1 = []
label_1 = []

train_2 = []
label_2 = []

train_3 = []
label_3 = []


for pose, label in zip(pose_values, label_values):
  if


# Combine the pose and label values into tuples


In [ ]:
from collections import Counter
element_count = Counter(label_values)
dictt = {}
# Print the frequency of each unique element
for element, count in element_count.items():
    print(f"Element {element} occurs {count} times.")
    dictt[element] = count
print(dictt)

Element clap occurs 31 times.
Element pour occurs 39 times.
Element run occurs 29 times.
Element wave occurs 30 times.
Element throw occurs 35 times.
Element golf occurs 30 times.
Element stand occurs 25 times.
Element sit occurs 27 times.
Element jump occurs 27 times.
Element pullup occurs 39 times.
Element pick occurs 28 times.
Element walk occurs 29 times.
Element push occurs 30 times.
Element catch occurs 34 times.
{'clap': 31, 'pour': 39, 'run': 29, 'wave': 30, 'throw': 35, 'golf': 30, 'stand': 25, 'sit': 27, 'jump': 27, 'pullup': 39, 'pick': 28, 'walk': 29, 'push': 30, 'catch': 34}


In [ ]:
# Desired count of samples per label in each part
from collections import defaultdict

label_counts = {'clap': 31, 'pour': 39, 'run': 29, 'wave': 30, 'throw': 35, 'golf': 30, 'stand': 25, 'sit': 27, 'jump': 27, 'pullup': 39, 'pick': 28, 'walk': 29, 'push': 30, 'catch': 34}






In [ ]:

# List of possible labels
possible_labels = ['pick', 'clap', 'jump', 'run', 'push', 'stand', 'pullup', 'throw', 'walk', 'catch', 'pour', 'wave', 'sit', 'golf']

# Create empty dictionaries for pose and label lists
pose_lists = {label: [] for label in possible_labels}
label_lists = {label: [] for label in possible_labels}

# Iterate over pose and label values
for pose, label in zip(pose_values, label_values):
    # Append pose value to the corresponding label list
    pose_lists[label].append(pose)
    # Append label value to the corresponding label list
    label_lists[label].append(label)

# Print the resulting pose and label lists for each label
for label in possible_labels:
    # print(f"Pose values for label '{label}': {pose_lists[label]}")
    # print(f"Label values for label '{label}': {label_lists[label]}")
    print()







In [ ]:
len(pose_lists['clap'])

31

In [ ]:
train1 = []
labels1 = []
train2 = []
labels2 = []
train3 = []
labels3 = []
for label in possible_labels:
    slice_size = label_counts[label] // 3

    train1.append(pose_lists[label][:slice_size])
    labels1.append(label_lists[label][:slice_size])
    train2.append(pose_lists[label][slice_size:2 * slice_size])
    labels2.append(label_lists[label][slice_size:2 * slice_size])
    train3.append(pose_lists[label][2 * slice_size:])
    labels3.append(label_lists[label][2 * slice_size:])




In [ ]:
new_Train = {'pose': [], 'label': []}



for train_class, label_class in zip(train1, labels1):
    for pose, label in zip(train_class, label_class):
      new_Train['pose'].append(pose)
      new_Train['label'].append(label)


for train_class, label_class in zip(train2, labels2):
    for pose, label in zip(train_class, label_class):
      new_Train['pose'].append(pose)
      new_Train['label'].append(label)

for train_class, label_class in zip(train3, labels3):
    for pose, label in zip(train_class, label_class):
      new_Train['pose'].append(pose)
      new_Train['label'].append(label)



In [ ]:
print(len(new_Train['label']))


433


In [ ]:
Train2 = pickle.load(open("/content/DD-Net-master/data/JHMDB/GT_train_2.pkl", "rb"))
Test2 = pickle.load(open("/content/DD-Net-master/data/JHMDB/GT_test_2.pkl", "rb"))

In [ ]:
Train3 = pickle.load(open("/content/DD-Net-master/data/JHMDB/GT_train_3.pkl", "rb"))
Test3 = pickle.load(open("/content/DD-Net-master/data/JHMDB/GT_test_3.pkl", "rb"))

In [ ]:
combined_dict = {}

# Iterate through dict1, dict2, and dict3
for dictionary in [Train1, Train2, Train3]:
    # Iterate through the keys in each dictionary
    for key, values in dictionary.items():
        if key not in combined_dict:
            # If the key doesn't exist in the combined dictionary, assign its value
            combined_dict[key] = values
        else:
            # If the key already exists, concatenate the values
            combined_dict[key] += values
Train = combined_dict

In [ ]:
combined_dict = {}

# Iterate through dict1, dict2, and dict3
for dictionary in [Test1, Test2, Test3]:
    # Iterate through the keys in each dictionary
    for key, values in dictionary.items():
        if key not in combined_dict:
            # If the key doesn't exist in the combined dictionary, assign its value
            combined_dict[key] = values
        else:
            # If the key already exists, concatenate the values
            combined_dict[key] += values
Test = combined_dict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


Load the data, in my case, it is KTH mediapipe keypoints in npy files

In [ ]:
import numpy as np


train = np.load('/content/drive/MyDrive/Abu_KTH/Mediapipe/train_poses.npy', allow_pickle=True)
val = np.load('/content/drive/MyDrive/Abu_KTH/Mediapipe/val_poses.npy', allow_pickle=True)
train = train.tolist()
val = val.tolist()

Also get the labels

In [ ]:
test = np.load('/content/drive/MyDrive/Abu_KTH/Mediapipe/test_poses.npy', allow_pickle=True)
test = test.tolist()
import pickle
test_videos = pickle.load(open("/content/drive/My Drive/KTH_Private/test.p", "rb"))
test_labels = []

for video in test_videos:
  test_labels.append(video['category'])
print(len(test_labels))


216


In [ ]:
train_videos = pickle.load(open("/content/drive/My Drive/KTH_Private/train.p", "rb"))
valid_videos = pickle.load(open("/content/drive/My Drive/KTH_Private/dev.p", "rb"))

In [ ]:
train_labels = []
for video in train_videos:
  train_labels.append(video['category'])
print(len(train_labels))

val_labels = []
for video in valid_videos:
  val_labels.append(video['category'])
print(len(val_labels))


191
192


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.datasets.cifar10 import load_data

In [ ]:
random.seed(1234)

class Config():
    def __init__(self):
        self.frame_l = 32 # the length of frames
        self.joint_n = 15 # the number of joints
        self.joint_d = 2 # the dimension of joints
        self.clc_num = 21 # the number of class
        self.feat_d = 105
        self.filters = 64
C = Config()

In [ ]:
# Temple resizing function
def zoom(p,target_l=64,joints_num=25,joints_dim=3):
    l = p.shape[0]
    p_new = np.empty([target_l,joints_num,joints_dim])
    for m in range(joints_num):
        for n in range(joints_dim):
            p[:,m,n] = medfilt(p[:,m,n],3)
            p_new[:,m,n] = inter.zoom(p[:,m,n],target_l/l)[:target_l]
    return p_new



# Calculate JCD feature
def norm_scale(x):
    return (x-np.mean(x))/np.mean(x)

def get_CG(p,C):
    M = []
    iu = np.triu_indices(C.joint_n,1,C.joint_n)
    for f in range(C.frame_l):
        d_m = cdist(p[f],p[f],'euclidean')
        d_m = d_m[iu]
        M.append(d_m)
    M = np.stack(M)
    M = norm_scale(M)
    return M


# Genrate dataset
def data_generator(T,C,le):
    X_0 = []
    X_1 = []
    Y = []
    for i in tqdm(range(len(T['pose']))):
        p = np.copy(T['pose'][i])
        p = zoom(p,target_l=C.frame_l,joints_num=C.joint_n,joints_dim=C.joint_d)

        label = np.zeros(C.clc_num)
        label[le.transform(T['label'])[i]-1] = 1

        M = get_CG(p,C)

        X_0.append(M)
        X_1.append(p)
        Y.append(label)

    X_0 = np.stack(X_0)
    X_1 = np.stack(X_1)
    Y = np.stack(Y)
    return X_0,X_1,Y

In [ ]:
def poses_diff(x):
    H, W = x.get_shape()[1],x.get_shape()[2]
    x = tf.subtract(x[:,1:,...],x[:,:-1,...])
    x = tf.image.resize(x,size=[H,W])
    return x

def pose_motion(P,frame_l):
    P_diff_slow = Lambda(lambda x: poses_diff(x))(P)
    P_diff_slow = Reshape((frame_l,-1))(P_diff_slow)
    P_fast = Lambda(lambda x: x[:,::2,...])(P)
    P_diff_fast = Lambda(lambda x: poses_diff(x))(P_fast)
    P_diff_fast = Reshape((int(frame_l/2),-1))(P_diff_fast)
    return P_diff_slow,P_diff_fast

def c1D(x,filters,kernel):
    x = Conv1D(filters, kernel_size=kernel,padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def block(x,filters):
    x = c1D(x,filters,3)
    x = c1D(x,filters,3)
    return x

def d1D(x,filters):
    x = Dense(filters,use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def build_FM(frame_l=32,joint_n=22,joint_d=2,feat_d=231,filters=16):
    M = Input(shape=(frame_l,feat_d))
    P = Input(shape=(frame_l,joint_n,joint_d))

    diff_slow,diff_fast = pose_motion(P,frame_l)

    x = c1D(M,filters*2,1)
    x = SpatialDropout1D(0.1)(x)
    x = c1D(x,filters,3)
    x = SpatialDropout1D(0.1)(x)
    x = c1D(x,filters,1)
    x = MaxPooling1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x_d_slow = c1D(diff_slow,filters*2,1)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
    x_d_slow = c1D(x_d_slow,filters,3)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
    x_d_slow = c1D(x_d_slow,filters,1)
    x_d_slow = MaxPool1D(2)(x_d_slow)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)

    x_d_fast = c1D(diff_fast,filters*2,1)
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)
    x_d_fast = c1D(x_d_fast,filters,3)
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)
    x_d_fast = c1D(x_d_fast,filters,1)
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)

    x = concatenate([x,x_d_slow,x_d_fast])
    x = block(x,filters*2)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x = block(x,filters*4)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x = block(x,filters*8)
    x = SpatialDropout1D(0.1)(x)

    return Model(inputs=[M,P],outputs=x)


def build_DD_Net(C):
    M = Input(name='M', shape=(C.frame_l,C.feat_d))
    P = Input(name='P', shape=(C.frame_l,C.joint_n,C.joint_d))

    FM = build_FM(C.frame_l,C.joint_n,C.joint_d,C.feat_d,C.filters)

    x = FM([M,P])

    x = GlobalMaxPool1D()(x)

    x = d1D(x,128)
    x = Dropout(0.5)(x)
    x = d1D(x,128)
    x = Dropout(0.5)(x)
    x = Dense(C.clc_num, activation='softmax')(x)

    ######################Self-supervised part
    model = Model(inputs=[M,P],outputs=x)
    return model

In [ ]:
DD_Net = build_DD_Net(C)
DD_Net.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 M (InputLayer)                 [(None, 32, 105)]    0           []                               
                                                                                                  
 P (InputLayer)                 [(None, 32, 15, 2)]  0           []                               
                                                                                                  
 model (Functional)             (None, 4, 512)       1714816     ['M[0][0]',                      
                                                                  'P[0][0]']                      
                                                                                                  
 global_max_pooling1d (GlobalMa  (None, 512)         0           ['model[0][0]']            

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Train['label'])

X_0,X_1,Y = data_generator(Train,C,le)
X_test_0,X_test_1,Y_test = data_generator(Test,C,le)

  0%|          | 0/433 [00:00<?, ?it/s]<ipython-input-9-e1ceada18105>:58: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  p_new[:,m,n] = inter.zoom(p[:,m,n],target_l/l)[:target_l]
<ipython-input-9-e1ceada18105>:58: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  p_new[:,m,n] = inter.zoom(p[:,m,n],target_l/l)[:target_l]
<ipython-input-9-e1ceada18105>:58: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  p_new[:,m,n] = inter.zoom(p[:,m,n],target_l/l)[:target_l]
<ipython-input-9-e1ceada18105>:58: DeprecationWarning: Please use `zoom` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  p_new[:,m,n] = inter.zoom(p[:,m,n],target_l/l)[:target_l]
<ipython-input-9-e1ceada18105>:58: DeprecationWar

In [ ]:
lr = 1e-3
DD_Net.compile(loss="categorical_crossentropy",optimizer=keras.optimizers.Adam(lr),metrics=['accuracy'])
lrScheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, cooldown=5, min_lr=5e-6)
history = DD_Net.fit([X_0,X_1],Y,
                    batch_size=len(Y),
                    epochs=600,
                    verbose=True,
                    shuffle=True,
                    callbacks=[lrScheduler],
                    validation_data=([X_test_0,X_test_1],Y_test)
                    )
lr = 1e-4
DD_Net.compile(loss="categorical_crossentropy",optimizer=keras.optimizers.Adam(lr),metrics=['accuracy'])
lrScheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, cooldown=5, min_lr=5e-6)
history = DD_Net.fit([X_0,X_1],Y,
                    batch_size=len(Y),
                    epochs=600,
                    verbose=True,
                    shuffle=True,
                    callbacks=[lrScheduler],
                    validation_data=([X_test_0,X_test_1],Y_test)
                    )

Epoch 1/600
1/1 [==============================] - 12s 12s/step - loss: 3.8172 - accuracy: 0.0624 - val_loss: 3.0408 - val_accuracy: 0.0682 - lr: 0.0010
Epoch 2/600
1/1 [==============================] - 4s 4s/step - loss: 3.2348 - accuracy: 0.1132 - val_loss: 3.0378 - val_accuracy: 0.0682 - lr: 0.0010
Epoch 3/600
1/1 [==============================] - 3s 3s/step - loss: 2.9232 - accuracy: 0.1709 - val_loss: 3.0340 - val_accuracy: 0.0739 - lr: 0.0010
Epoch 4/600
1/1 [==============================] - 3s 3s/step - loss: 2.7165 - accuracy: 0.2494 - val_loss: 3.0274 - val_accuracy: 0.0795 - lr: 0.0010
Epoch 5/600
1/1 [==============================] - 3s 3s/step - loss: 2.5163 - accuracy: 0.2587 - val_loss: 3.0184 - val_accuracy: 0.1364 - lr: 0.0010
Epoch 6/600
1/1 [==============================] - 3s 3s/step - loss: 2.2712 - accuracy: 0.3418 - val_loss: 3.0091 - val_accuracy: 0.1989 - lr: 0.0010
Epoch 7/600
1/1 [==============================] - 5s 5s/step - loss: 2.1055 - accuracy: 0.3

In [ ]:
print(len(Train['label']))



433


In [ ]:
def poses_diff(x):
    H, W = x.get_shape()[1],x.get_shape()[2]
    x = tf.subtract(x[:,1:,...],x[:,:-1,...])

    x = tf.image.resize(x,size=[H,W])
    return x


In [ ]:
import numpy as np
import random
from tqdm import tqdm
import scipy.ndimage.interpolation as inter
from scipy.signal import medfilt
from scipy.spatial.distance import cdist

from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.layers.core import *
from tensorflow.keras.callbacks import *
from keras.layers.convolutional import *
import tensorflow as tf

import numpy as np
import math
import random
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import glob
import gc

from tqdm import tqdm
import pickle

from keras.optimizers import *
from keras.models import Model
from keras.layers import *
from keras.layers.core import *
from keras.layers.convolutional import *
from keras import backend as K
import keras
import tensorflow as tf

#Initialize the setting
random.seed(1234)

class Config():
    def __init__(self):
        self.frame_l = 32 # the length of frames
        self.joint_n = 15 # the number of joints
        self.joint_d = 2 # the dimension of joints
        self.clc_num = 21 # the number of class
        self.feat_d = 105
        self.filters = 64


# Temple resizing function
def zoom(p,target_l=64,joints_num=33,joints_dim=2):
    l = p.shape[0]
    p_new = np.empty([target_l,joints_num,joints_dim])
    for m in range(joints_num):
        for n in range(joints_dim):
            p[:,m,n] = medfilt(p[:,m,n],3)
            p_new[:,m,n] = inter.zoom(p[:,m,n],target_l/l)[:target_l]
    return p_new

# Calculate JCD feature
def norm_scale(x):
    return (x-np.mean(x))/np.mean(x)

def get_CG(p,C):
    M = []
    iu = np.triu_indices(C.joint_n,1,C.joint_n)
    for f in range(C.frame_l):
        d_m = cdist(p[f],p[f],'euclidean')
        d_m = d_m[iu]
        M.append(d_m)
    M = np.stack(M)
    M = norm_scale(M)
    return M


def data_generator(T,C,le):
    X_0 = []
    X_1 = []
    Y = []
    for i in tqdm(range(len(T['pose']))):
        p = np.copy(T['pose'][i])
        p = zoom(p,target_l=C.frame_l,joints_num=C.joint_n,joints_dim=C.joint_d)

        label = np.zeros(C.clc_num)
        label[le.transform(T['label'])[i]-1] = 1

        M = get_CG(p,C)

        X_0.append(M)
        X_1.append(p)
        Y.append(label)

    X_0 = np.stack(X_0)
    X_1 = np.stack(X_1)
    Y = np.stack(Y)
    return X_0,X_1,Y




#Building the model
# def poses_diff(x):
#     H, W = x.get_shape()[1],x.get_shape()[2]
#     x = tf.subtract(x[:,1:,...],x[:,:-1,...])
#     x = tf.image.resize(x,size=[H,W])
#     return x

# def pose_motion(P,frame_l):
#     P_diff_slow = Lambda(lambda x: poses_diff(x))(P)
#     P_diff_slow = Reshape((frame_l,-1))(P_diff_slow)
#     P_fast = Lambda(lambda x: x[:,::2,...])(P)
#     P_diff_fast = Lambda(lambda x: poses_diff(x))(P_fast)
#     P_diff_fast = Reshape((int(frame_l/2),-1))(P_diff_fast)
#     return P_diff_slow,P_diff_fast

# def pose_motion(P, frame_l):
#     def pose_diff(x):
#         H, W = x.get_shape()[1], x.get_shape()[2]
#         x = tf.subtract(x[:, 1:, ...], x[:, :-1, ...])
#         x = tf.image.resize(x, size=[H, W])
#         return x

#     P_diff_slow = Lambda(lambda x: pose_diff(x))(P)
#     P_diff_slow = Reshape((frame_l, -1))(P_diff_slow)
#     P_fast = Lambda(lambda x: x[:, ::2, ...])(P)
#     P_diff_fast = Lambda(lambda x: pose_diff(x))(P_fast)
#     P_diff_fast = Reshape((int(frame_l/2), -1))(P_diff_fast)
#     return P_diff_slow, P_diff_fast


# def pose_motion(P, frame_l):
#     P_diff_slow = Lambda(lambda x: tf.subtract(x[:, 1:, ...], x[:, :-1, ...]))(P)
#     P_diff_slow = Reshape((frame_l, -1))(P_diff_slow)
#     P_fast = Lambda(lambda x: x[:, ::2, ...])(P)
#     P_diff_fast = Lambda(lambda x: tf.subtract(x[:, 1:, ...], x[:, :-1, ...]))(P_fast)
#     P_diff_fast = Reshape((int(frame_l/2), -1))(P_diff_fast)
#     return P_diff_slow, P_diff_fast


# def pose_motion(P, frame_l):
#     P_diff_slow = Lambda(lambda x: tf.subtract(x[:, 1:, ...], x[:, :-1, ...]))(P)
#     P_diff_slow = Reshape((frame_l, -1))(P_diff_slow)
#     P_fast = Lambda(lambda x: x[:, ::2, ...])(P)
#     P_diff_fast = Lambda(lambda x: tf.subtract(x[:, 1:, ...], x[:, :-1, ...]))(P_fast)
#     P_diff_fast = Reshape((int(frame_l/2), -1))(P_diff_fast)
#     return P_diff_slow, P_diff_fast

def pose_motion(P, frame_l):
    P_diff_slow = Lambda(lambda x: tf.subtract(x[:, 1:, ...], x[:, :-1, ...]))(P)
    P_diff_slow = Reshape((frame_l-1, -1))(P_diff_slow)
    P_fast = Lambda(lambda x: x[:, :frame_l-1:2, ...])(P)
    P_diff_fast = Lambda(lambda x: tf.subtract(x[:, 1:, ...], x[:, :-1, ...]))(P_fast)
    P_diff_fast = Reshape((int((frame_l-1)/2), -1))(P_diff_fast)
    return P_diff_slow, P_diff_fast





def c1D(x,filters,kernel):
    x = Conv1D(filters, kernel_size=kernel,padding='same',use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def block(x,filters):
    x = c1D(x,filters,3)
    x = c1D(x,filters,3)
    return x

def d1D(x,filters):
    x = Dense(filters,use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    return x

def build_FM(frame_l=30,joint_n=33,joint_d=2,feat_d=231,filters=16):
    M = Input(shape=(frame_l,feat_d))
    P = Input(shape=(frame_l,joint_n,joint_d))

    diff_slow,diff_fast = pose_motion(P,frame_l)

    x = c1D(M,filters*2,1)
    x = SpatialDropout1D(0.1)(x)
    x = c1D(x,filters,3)
    x = SpatialDropout1D(0.1)(x)
    x = c1D(x,filters,1)
    x = MaxPooling1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x_d_slow = c1D(diff_slow,filters*2,1)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
    x_d_slow = c1D(x_d_slow,filters,3)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)
    x_d_slow = c1D(x_d_slow,filters,1)
    x_d_slow = MaxPool1D(2)(x_d_slow)
    x_d_slow = SpatialDropout1D(0.1)(x_d_slow)

    x_d_fast = c1D(diff_fast,filters*2,1)
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)
    x_d_fast = c1D(x_d_fast,filters,3)
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)
    x_d_fast = c1D(x_d_fast,filters,1)
    x_d_fast = SpatialDropout1D(0.1)(x_d_fast)

    # x = concatenate([x,x_d_slow,x_d_fast])
    x = concatenate([x[:, :-1, :], x_d_slow, x_d_fast])
    x = block(x,filters*2)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x = block(x,filters*4)
    x = MaxPool1D(2)(x)
    x = SpatialDropout1D(0.1)(x)

    x = block(x,filters*8)
    x = SpatialDropout1D(0.1)(x)

    return Model(inputs=[M,P],outputs=x)


def build_DD_Net(C):
    M = Input(name='M', shape=(C.frame_l,C.feat_d))
    P = Input(name='P', shape=(C.frame_l,C.joint_n,C.joint_d))

    FM = build_FM(C.frame_l,C.joint_n,C.joint_d,C.feat_d,C.filters)

    x = FM([M,P])

    x = GlobalMaxPool1D()(x)

    x = d1D(x,128)
    x = Dropout(0.5)(x)
    x = d1D(x,128)
    x = Dropout(0.5)(x)
    x = Dense(C.clc_num, activation='softmax')(x)

    ######################Self-supervised part
    model = Model(inputs=[M,P],outputs=x)
    return model


#Test in real time
# def data_generator_rt(T,C):
#     X_0 = []
#     X_1 = []

#     T = np.expand_dims(T, axis = 0)
#     for i in tqdm(range(len(T))):
#         p = np.copy(T[i])
#         p = zoom(p,target_l=C.frame_l,joints_num=C.joint_n,joints_dim=C.joint_d)

#         M = get_CG(p,C)

#         X_0.append(M)
#         X_1.append(p)

#     X_0 = np.stack(X_0)
#     X_1 = np.stack(X_1)

#     return X_0,X_1

In [ ]:
C = Config()
DD_Net = build_DD_Net(C)
DD_Net.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 M (InputLayer)                 [(None, 32, 105)]    0           []                               
                                                                                                  
 P (InputLayer)                 [(None, 32, 15, 2)]  0           []                               
                                                                                                  
 model_2 (Functional)           (None, 3, 512)       1714816     ['M[0][0]',                      
                                                                  'P[0][0]']                      
                                                                                                  
 global_max_pooling1d_1 (Global  (None, 512)         0           ['model_2[0][0]']          

Create X_0, X_1, Y for the training using Data generator (for dd_net only, please refer to the original tutorial for general version)


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(train_labels)


X_0,X_1,Y = data_generator(train,C,le, train_labels)
X_val_0,X_val_1,Y_val = data_generator(val,C, le, val_labels)

NameError: ignored

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(new_Train['label'])

X_0,X_1,Y = data_generator(new_Train,C,le)
X_test_0,X_test_1,Y_test = data_generator(Test,C,le)

NameError: ignored

In [ ]:
Y.shape

(191, 6)

In [ ]:
type(X_0)

numpy.ndarray

In [ ]:
X_1.shape

(433, 32, 15, 2)

In [ ]:
DD_Net.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
class CifarClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader


    def get_parameters(self, config):
        return DD_Net.get_weights()

    def fit(self, parameters, config):
        DD_Net.set_weights(parameters)
        DD_Net.fit(self.trainloader[0], self.trainloader[1], epochs=80, batch_size=len(self.trainloader[1] ))
        return DD_Net.get_weights(), len(self.trainloader[1]), {}

    def evaluate(self, parameters, config):
        DD_Net.set_weights(parameters)
        loss, accuracy = DD_Net.evaluate(self.valloader[0], self.valloader[1])
        print("ACCURACYYY:", accuracy)
        return loss, len(self.valloader[1]), {"accuracy": float(accuracy)}

In [ ]:
X_test_0, X_test_1, Y_test = X_val_0, X_val_1, Y_val

In [ ]:
def client_fn(cid: str) -> CifarClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = DD_Net

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = ([X_0[int(cid)*144:int(cid)*144+144], X_1[int(cid)*144:int(cid)*144+144]], Y[int(cid)*144:int(cid)*144+144])
    valloader = ([X_test_0, X_test_1], Y_test)

    # Create a  single Flower client representing a single organization
    return CifarClient(net, trainloader, valloader)

In [ ]:
# Create FedAvg strategy
#jhmdb non-iid

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1,  # Sample 50% of available clients for evaluation
    min_fit_clients=3,  # Never sample less than 10 clients for training
    min_evaluate_clients=3,  # Never sample less than 5 clients for evaluation
    min_available_clients=3,  # Wait until all 10 clients are available
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None


# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=15),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-22 16:34:58,009 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


1/1 [==============================] - 3s 3s/step - loss: 0.0038 - accuracy: 1.0000 [repeated 2x across cluster]
(launch_and_fit pid=43159) Epoch 41/80 [repeated 3x across cluster]


(launch_and_fit pid=43160) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
2023-06-22 16:35:05,727	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-22 16:35:08,523 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 7777289012.0, 'object_store_memory': 3888644505.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7777289012.0, 'object_store_memory': 3888644505.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-06-22 16:35:08,532 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-22 16:35:08,538 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=57120) 2023-06-22 16:35:12.095772: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fi

(launch_and_fit pid=57120) Epoch 1/80


(launch_and_fit pid=57119) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


...

(launch_and_evaluate pid=57119) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
(launch_and_evaluate pid=57120) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


6/6 [==============================] - 3s 34ms/step - loss: 1.4011 - accuracy: 0.6364
(launch_and_evaluate pid=57119) ACCURACYYY: 0.6363636255264282


(launch_and_evaluate pid=57119) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
DEBUG flwr 2023-06-22 18:14:39,422 | server.py:182 | evaluate_round 15 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 15 received 3 results and 0 failures
INFO flwr 2023-06-22 18:14:39,428 | server.py:147 | FL finished in 5962.686056736
INFO:flwr:FL finished in 5962.686056736
INFO flwr 2023-06-22 18:14:39,431 | app.py:218 | app_fit: losses_distributed [(1, 2.9365732669830322), (2, 2.7371129989624023), (3, 2.5351645946502686), (4, 2.36802077293396), (5, 2.3041343688964844), (6, 1.6954560279846191), (7, 2.217153787612915), (8, 1.285926103591919), (9, 1.5375094413757324), (10, 1.0488909482955933), (11, 1.0856900215148926), (12, 1.0326881408691406), (13, 1.2220149040222168), (14, 0.9744850397109985), (15, 1.4010958671569824)]
INFO:flwr:app_fit: losses_distributed [(1, 2.9365732669830322), (2, 2.7371129989624023)

6/6 [==============================] - 2s 27ms/step - loss: 1.4011 - accuracy: 0.6364


History (loss, distributed):
	round 1: 2.9365732669830322
	round 2: 2.7371129989624023
	round 3: 2.5351645946502686
	round 4: 2.36802077293396
	round 5: 2.3041343688964844
	round 6: 1.6954560279846191
	round 7: 2.217153787612915
	round 8: 1.285926103591919
	round 9: 1.5375094413757324
	round 10: 1.0488909482955933
	round 11: 1.0856900215148926
	round 12: 1.0326881408691406
	round 13: 1.2220149040222168
	round 14: 0.9744850397109985
	round 15: 1.4010958671569824

In [ ]:
#jhmdb iid
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1,  # Sample 50% of available clients for evaluation
    min_fit_clients=3,  # Never sample less than 10 clients for training
    min_evaluate_clients=3,  # Never sample less than 5 clients for evaluation
    min_available_clients=3,  # Wait until all 10 clients are available
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None


# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=15),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-22 21:08:39,496 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


6/6 [==============================] - 3s 38ms/step - loss: 1.4011 - accuracy: 0.6364
(launch_and_evaluate pid=57119) ACCURACYYY: 0.6363636255264282 [repeated 2x across cluster]


2023-06-22 21:08:47,682	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-22 21:08:51,551 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 3888993484.0, 'memory': 7777986971.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 3888993484.0, 'memory': 7777986971.0}
INFO flwr 2023-06-22 21:08:51,563 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-22 21:08:51,569 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=133981) 2023-06-22 21:08:55.430381: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(launch_and_get_parameters pid=133981) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved opt

(launch_and_fit pid=133981) Epoch 1/80


(launch_and_fit pid=133980) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


...

(launch_and_evaluate pid=133980) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
(launch_and_evaluate pid=133981) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


6/6 [==============================] - 2s 39ms/step - loss: 2.2714 - accuracy: 0.7841
(launch_and_evaluate pid=133980) ACCURACYYY: 0.7840909361839294


DEBUG flwr 2023-06-22 22:50:28,732 | server.py:182 | evaluate_round 15 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 15 received 3 results and 0 failures
INFO flwr 2023-06-22 22:50:28,742 | server.py:147 | FL finished in 6087.8309026789975
INFO:flwr:FL finished in 6087.8309026789975
INFO flwr 2023-06-22 22:50:28,747 | app.py:218 | app_fit: losses_distributed [(1, 3.3865487575531006), (2, 4.988807201385498), (3, 5.703599452972412), (4, 4.900888442993164), (5, 3.4981141090393066), (6, 3.027087450027466), (7, 2.3894777297973633), (8, 1.9658875465393066), (9, 2.1024668216705322), (10, 1.634164810180664), (11, 1.3836427927017212), (12, 1.7631099224090576), (13, 1.5974549055099487), (14, 1.553151249885559), (15, 2.2713992595672607)]
INFO:flwr:app_fit: losses_distributed [(1, 3.3865487575531006), (2, 4.988807201385498), (3, 5.703599452972412), (4, 4.900888442993164), (5, 3.4981141090393066), (6, 3.027087450027466), (7, 2.3894777297973633), (8, 1.9658875465393066), (9, 2.10246682

4/6 [===================>..........] - ETA: 0s - loss: 2.0528 - accuracy: 0.7969


History (loss, distributed):
	round 1: 3.3865487575531006
	round 2: 4.988807201385498
	round 3: 5.703599452972412
	round 4: 4.900888442993164
	round 5: 3.4981141090393066
	round 6: 3.027087450027466
	round 7: 2.3894777297973633
	round 8: 1.9658875465393066
	round 9: 2.1024668216705322
	round 10: 1.634164810180664
	round 11: 1.3836427927017212
	round 12: 1.7631099224090576
	round 13: 1.5974549055099487
	round 14: 1.553151249885559
	round 15: 2.2713992595672607

In [ ]:
#jhmdb iid
# Create FedProx strategy
strategy = fl.server.strategy.FedProx(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1,  # Sample 50% of available clients for evaluation
    min_fit_clients=3,  # Never sample less than 10 clients for training
    min_evaluate_clients=3,  # Never sample less than 5 clients for evaluation
    min_available_clients=3,  # Wait until all 10 clients are available
    proximal_mu = 0.001,
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None


# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=15),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-22 22:55:15,622 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


6/6 [==============================] - 2s 48ms/step - loss: 2.2714 - accuracy: 0.7841 [repeated 2x across cluster]
(launch_and_evaluate pid=133980) ACCURACYYY: 0.7840909361839294 [repeated 2x across cluster]


(launch_and_evaluate pid=133980) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
2023-06-22 22:55:24,919	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-22 22:55:27,614 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 7788768462.0, 'object_store_memory': 3894384230.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7788768462.0, 'object_store_memory': 3894384230.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO flwr 2023-06-22 22:55:27,621 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-22 22:55:27,624 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=168341) 2023-06-22 22:55:30.869572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could

(launch_and_fit pid=168341) Epoch 1/80


(launch_and_fit pid=168340) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


...

DEBUG flwr 2023-06-23 00:32:18,616 | server.py:232 | fit_round 15 received 3 results and 0 failures
DEBUG:flwr:fit_round 15 received 3 results and 0 failures
DEBUG flwr 2023-06-23 00:32:18,717 | server.py:168 | evaluate_round 15: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 15: strategy sampled 3 clients (out of 3)
(launch_and_evaluate pid=168341) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
(launch_and_evaluate pid=168340) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


6/6 [==============================] - 4s 61ms/step - loss: 1.7320 - accuracy: 0.8068
(launch_and_evaluate pid=168341) ACCURACYYY: 0.8068181872367859
4/6 [===================>..........] - ETA: 0s - loss: 1.3874 - accuracy: 0.8281


(launch_and_evaluate pid=168341) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
DEBUG flwr 2023-06-23 00:32:31,913 | server.py:182 | evaluate_round 15 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 15 received 3 results and 0 failures
INFO flwr 2023-06-23 00:32:31,922 | server.py:147 | FL finished in 5816.217496778998
INFO:flwr:FL finished in 5816.217496778998
INFO flwr 2023-06-23 00:32:31,925 | app.py:218 | app_fit: losses_distributed [(1, 3.164486885070801), (2, 5.226729869842529), (3, 6.804122447967529), (4, 4.212151527404785), (5, 3.0122666358947754), (6, 2.599142551422119), (7, 2.000344753265381), (8, 2.161003351211548), (9, 1.722076416015625), (10, 1.9429997205734253), (11, 2.0101068019866943), (12, 1.8952304124832153), (13, 1.7424052953720093), (14, 1.4742556810379028), (15, 1.7319881916046143)]
INFO:flwr:app_fit: losses_distributed [(1, 3.164486885070801), (2, 5.226729869842529

6/6 [==============================] - 2s 22ms/step - loss: 1.7320 - accuracy: 0.8068
(launch_and_evaluate pid=168341) ACCURACYYY: 0.8068181872367859


History (loss, distributed):
	round 1: 3.164486885070801
	round 2: 5.226729869842529
	round 3: 6.804122447967529
	round 4: 4.212151527404785
	round 5: 3.0122666358947754
	round 6: 2.599142551422119
	round 7: 2.000344753265381
	round 8: 2.161003351211548
	round 9: 1.722076416015625
	round 10: 1.9429997205734253
	round 11: 2.0101068019866943
	round 12: 1.8952304124832153
	round 13: 1.7424052953720093
	round 14: 1.4742556810379028
	round 15: 1.7319881916046143

In [ ]:
#jhmdb non-iid
# Create FedAvg strategy
strategy = fl.server.strategy.FedProx(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1,  # Sample 50% of available clients for evaluation
    min_fit_clients=3,  # Never sample less than 10 clients for training
    min_evaluate_clients=3,  # Never sample less than 5 clients for evaluation
    min_available_clients=3,  # Wait until all 10 clients are available
    proximal_mu = 0.001,
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None


# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=3,
    config=fl.server.ServerConfig(num_rounds=15),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-23 01:54:17,916 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)
2023-06-23 01:54:22,841	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-23 01:54:25,897 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7811447195.0, 'object_store_memory': 3905723596.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7811447195.0, 'object_store_memory': 3905723596.0}
INFO flwr 2023-06-23 01:54:25,908 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-23 01:54:25,912 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=2747) 2023-06-23 01:54:29.359859: W tensorflow/compiler

(launch_and_fit pid=2747) Epoch 1/80


(launch_and_fit pid=2746) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


(launch_and_fit pid=2746) Epoch 1/80
1/1 [==============================] - 18s 18s/step - loss: 4.0499 - accuracy: 0.0278
(launch_and_fit pid=2747) Epoch 2/80
1/1 [==============================] - 2s 2s/step - loss: 3.2636 - accuracy: 0.1597
(launch_and_fit pid=2747) Epoch 3/80
1/1 [==============================] - 2s 2s/step - loss: 2.8287 - accuracy: 0.1875
(launch_and_fit pid=2747) Epoch 4/80
1/1 [==============================] - 3s 3s/step - loss: 2.4833 - accuracy: 0.2778
(launch_and_fit pid=2747) Epoch 5/80
1/1 [==============================] - 2s 2s/step - loss: 2.3469 - accuracy: 0.3125
(launch_and_fit pid=2747) Epoch 6/80
1/1 [==============================] - 3s 3s/step - loss: 2.9688 - accuracy: 0.1875 [repeated 5x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(launch_and_fit pid=2746) Epoch 4/80 [repeated 5

(launch_and_fit pid=2747) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


...

DEBUG flwr 2023-06-23 03:22:28,918 | server.py:232 | fit_round 15 received 3 results and 0 failures
DEBUG:flwr:fit_round 15 received 3 results and 0 failures
DEBUG flwr 2023-06-23 03:22:29,048 | server.py:168 | evaluate_round 15: strategy sampled 3 clients (out of 3)
DEBUG:flwr:evaluate_round 15: strategy sampled 3 clients (out of 3)
(launch_and_evaluate pid=2747) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 
(launch_and_evaluate pid=2746) WARNING:absl:Skipping variable loading for optimizer 'Adam', because it has 107 variables whereas the saved optimizer has 1 variables. 


1/1 [==============================] - 1s 1s/step - loss: 2.7873e-04 - accuracy: 1.0000 [repeated 3x across cluster]
(launch_and_fit pid=2746) Epoch 80/80 [repeated 2x across cluster]
6/6 [==============================] - 2s 35ms/step - loss: 1.0647 - accuracy: 0.6932
(launch_and_evaluate pid=2747) ACCURACYYY: 0.6931818127632141


DEBUG flwr 2023-06-23 03:22:41,776 | server.py:182 | evaluate_round 15 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 15 received 3 results and 0 failures
INFO flwr 2023-06-23 03:22:41,782 | server.py:147 | FL finished in 5288.322435529
INFO:flwr:FL finished in 5288.322435529
INFO flwr 2023-06-23 03:22:41,788 | app.py:218 | app_fit: losses_distributed [(1, 2.951580047607422), (2, 2.833294153213501), (3, 2.735687732696533), (4, 2.1236963272094727), (5, 2.382143020629883), (6, 1.7379047870635986), (7, 1.5230721235275269), (8, 1.3151493072509766), (9, 1.2928391695022583), (10, 1.1524410247802734), (11, 1.4528396129608154), (12, 1.1118181943893433), (13, 1.225797176361084), (14, 1.2326925992965698), (15, 1.0647114515304565)]
INFO:flwr:app_fit: losses_distributed [(1, 2.951580047607422), (2, 2.833294153213501), (3, 2.735687732696533), (4, 2.1236963272094727), (5, 2.382143020629883), (6, 1.7379047870635986), (7, 1.5230721235275269), (8, 1.3151493072509766), (9, 1.292839169502258

5/6 [========================>.....] - ETA: 0s - loss: 1.0073 - accuracy: 0.7000


History (loss, distributed):
	round 1: 2.951580047607422
	round 2: 2.833294153213501
	round 3: 2.735687732696533
	round 4: 2.1236963272094727
	round 5: 2.382143020629883
	round 6: 1.7379047870635986
	round 7: 1.5230721235275269
	round 8: 1.3151493072509766
	round 9: 1.2928391695022583
	round 10: 1.1524410247802734
	round 11: 1.4528396129608154
	round 12: 1.1118181943893433
	round 13: 1.225797176361084
	round 14: 1.2326925992965698
	round 15: 1.0647114515304565